In [20]:
import glob
import pandas as pd
import soundfile as sf
from tqdm import tqdm 
import re

wav_path = './data/cnceleb/data/'
wav_list = './data/cnceleb_train_wav.txt'
vad_path = './data/cnceleb_vad/'
vad_list = './data/cnceleb_train_vad.txt'


def get_len(p, plist):
    with open(plist, 'r') as f:
        files = f.readlines()
    l = []
    for f in tqdm(files):
        f = f.split(' ')[1]        
        audio, _ = sf.read(p + f.strip())
        l.append(len(audio))
    return l

# wavlen = get_len(wav_path, wav_list)
vadlen = get_len(vad_path, vad_list)
    


  5%|▍         | 3321/69939 [00:03<01:00, 1100.39it/s]


KeyboardInterrupt: 

In [34]:
import glob 

dev_list = []
with open('./data/cnceleb/dev/dev.lst') as f:
    for line in f:
        dev_list.append(line.strip())


files = glob.glob('./data/cnceleb/data/*/*.flac')
c = 0
for file in files:
    if 'concat' in file: continue        
    curr_label = re.findall(r'(id\d+)', file)[0]
    if curr_label not in dev_list: continue
    c+=1
print(c)

107953


In [36]:
import glob 

dev_list = []
with open('./data/cnceleb/dev/dev.lst') as f:
    for line in f:
        dev_list.append(line.strip())


files = glob.glob('./data/cnceleb_wav/data/*/*.wav')
c = 0
for file in files:
    if 'concat' in file: continue        
    curr_label = re.findall(r'(id\d+)', file)[0]
    if curr_label not in dev_list: continue
    c+=1
print(c)

107953


In [ ]:
53288 - 49822

In [19]:
import pandas as pd 

wav_df = pd.Series(wavlen)
print(wav_df.describe())

vad_df = pd.Series(vadlen)
print(vad_df.describe())

count    7.985700e+04
mean     1.776973e+05
std      2.753283e+05
min      5.051800e+04
25%      8.507800e+04
50%      1.107130e+05
75%      1.746150e+05
max      3.644678e+07
dtype: float64
count    7.846100e+04
mean     1.696714e+05
std      2.553896e+05
min      4.896000e+04
25%      8.256000e+04
50%      1.075200e+05
75%      1.680000e+05
max      3.339504e+07
dtype: float64


In [29]:
from pathlib import PurePath
from pydub import AudioSegment
from tqdm import tqdm
from scipy.io import wavfile

import glob
import multiprocessing
import os
import re
import webrtcvad
import struct
import argparse
import shutil

import numpy as np
import soundfile as sf

dev_list = []
with open('./data/cnceleb/dev/dev.lst') as f:
    for line in f:
        dev_list.append(line.strip())

files = glob.glob('./data/cnceleb_vad/id00000/*.wav')
tlen = 400 * 160 + 240 # concat short utterances (<400frames)

cat = np.array([])
curr_label = None
prev_label = None

print('concat files...')
for i,file in enumerate(tqdm(files)):

    if 'concat' in file: 
        os.unlink(file)
        continue

    curr_label = re.findall(r'(id\d+)', file)[0]
    if curr_label not in dev_list: continue

    _, audio = wavfile.read(file)

    if curr_label != prev_label and len(cat) > 0:
        pure_path = PurePath(file)
        fname = f'{os.path.split(pure_path)[0]}/{curr_label}-concat-{i}{pure_path.suffix}'
        wavfile.write(fname, rate = 16000, data = cat)
        cat = np.array([])
        break

    if len(audio) < tlen:
        print(file, len(audio), tlen)
        cat = np.concatenate([cat, audio])

    prev_label = curr_label

concat files...


100%|██████████| 7/7 [00:00<00:00, 361.35it/s]


In [30]:
files = glob.glob('./data/cnceleb_vad/*/*.wav')

In [31]:
files[:100]

['./data/cnceleb_vad/id00661/singing-02-021.wav',
 './data/cnceleb_vad/id00661/singing-03-011.wav',
 './data/cnceleb_vad/id00661/singing-02-004.wav',
 './data/cnceleb_vad/id00661/singing-02-017.wav',
 './data/cnceleb_vad/id00661/singing-02-009.wav',
 './data/cnceleb_vad/id00661/singing-02-011.wav',
 './data/cnceleb_vad/id00661/singing-03-001.wav',
 './data/cnceleb_vad/id00661/singing-03-018.wav',
 './data/cnceleb_vad/id00661/singing-03-020.wav',
 './data/cnceleb_vad/id00661/singing-02-016.wav',
 './data/cnceleb_vad/id00661/singing-02-005.wav',
 './data/cnceleb_vad/id00661/singing-03-008.wav',
 './data/cnceleb_vad/id00661/singing-02-008.wav',
 './data/cnceleb_vad/id00661/singing-03-019.wav',
 './data/cnceleb_vad/id00661/singing-03-027.wav',
 './data/cnceleb_vad/id00661/vlog-01-005.wav',
 './data/cnceleb_vad/id00661/singing-03-029.wav',
 './data/cnceleb_vad/id00661/singing-02-014.wav',
 './data/cnceleb_vad/id00661/singing-03-010.wav',
 './data/cnceleb_vad/id00661/singing-03-023.wav',
 '.